## Setting up the environment (data + libraries)

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# read the dataset
train = pd.read_csv('data/training_set.csv')

## Skimming through the data

In [3]:
# try to extract some data instances
train.head()

,Unnamed: 0,msno,is_churn,city,bd,gender,registered_via,TimeSinceReg,is_auto_renew_median,is_auto_renew_last,...,total_secs_avg_3mo,count_3mo,num_25_avg_6mo,num_50_avg_6mo,num_75_avg_6mo,num_985_avg_6mo,num_100_avg_6mo,num_unq_avg_6mo,total_secs_avg_6mo,count_6mo
0,0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,18.0,36.0,2.0,9.0,4346 days 00:00:00.000000000,0.0,0.0,...,4613.960400,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,10.0,38.0,1.0,9.0,4345 days 00:00:00.000000000,1.0,1.0,...,10587.551900,10.0,0.619048,0.214286,0.047619,0.238095,5.119048,3.357143,1394.373071,42.0
2,2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,11.0,27.0,2.0,9.0,4153 days 00:00:00.000000000,1.0,1.0,...,5601.753895,19.0,5.206349,2.746032,1.079365,2.015873,34.809524,42.000000,8994.666714,63.0
3,3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,13.0,23.0,2.0,9.0,4136 days 00:00:00.000000000,0.0,0.0,...,5940.487118,76.0,9.034884,2.674419,1.511628,1.395349,39.232558,32.255814,9027.646605,86.0
4,4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,3.0,27.0,1.0,9.0,4080 days 00:00:00.000000000,0.0,0.0,...,22933.773487,76.0,1.865169,0.831461,0.730337,1.775281,87.449438,89.617978,22925.477517,89.0


As seen from the sample of 5 extracted rows:
* 46 features
* NULL values are denoted as NaN (and not -1 as in Porto Segure competition)

In [4]:
train.columns.values # all the features of training set

array(['Unnamed: 0', 'msno', 'is_churn', 'city', 'bd', 'gender',
       'registered_via', 'TimeSinceReg', 'is_auto_renew_median',
       'is_auto_renew_last', 'membership_expire_date_last',
       'transaction_date_last', 'payment_plan_days_mean',
       'payment_plan_days_last', 'actual_amount_paid_mean',
       'actual_amount_paid_last', 'plan_list_price_mean',
       'plan_list_price_last', 'is_cancel_mean', 'is_cancel_last',
       'payment_method_id_mean', 'payment_method_id_last',
       'num_25_avg_1mo', 'num_50_avg_1mo', 'num_75_avg_1mo',
       'num_985_avg_1mo', 'num_100_avg_1mo', 'num_unq_avg_1mo',
       'total_secs_avg_1mo', 'count_1mo', 'num_25_avg_3mo',
       'num_50_avg_3mo', 'num_75_avg_3mo', 'num_985_avg_3mo',
       'num_100_avg_3mo', 'num_unq_avg_3mo', 'total_secs_avg_3mo',
       'count_3mo', 'num_25_avg_6mo', 'num_50_avg_6mo', 'num_75_avg_6mo',
       'num_985_avg_6mo', 'num_100_avg_6mo', 'num_unq_avg_6mo',
       'total_secs_avg_6mo', 'count_6mo'], dtype=object)

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992931 entries, 0 to 992930
Data columns (total 46 columns):
Unnamed: 0                     992931 non-null int64
msno                           992931 non-null object
is_churn                       992931 non-null int64
city                           877161 non-null float64
bd                             877161 non-null float64
gender                         992931 non-null float64
registered_via                 877161 non-null float64
TimeSinceReg                   877161 non-null object
is_auto_renew_median           990834 non-null float64
is_auto_renew_last             990834 non-null float64
membership_expire_date_last    990834 non-null float64
transaction_date_last          990834 non-null float64
payment_plan_days_mean         990834 non-null float64
payment_plan_days_last         990834 non-null float64
actual_amount_paid_mean        990834 non-null float64
actual_amount_paid_last        990834 non-null float64
plan_list_price

From the above summary:
* total rows: **992,931**
* types of features:
    * **categorical**: city, bd, gender, registered_via, is_auto_renew_median, is_auto_renew_last, plan_list_price_mean, plan_list_price_last, is_cancel_mean, is_cancel_last
    * **object**: TimeSinceReg, msno
    * **numerical**: all the other features

In [6]:
train.describe()

,Unnamed: 0,is_churn,city,bd,gender,registered_via,is_auto_renew_median,is_auto_renew_last,membership_expire_date_last,transaction_date_last,...,total_secs_avg_3mo,count_3mo,num_25_avg_6mo,num_50_avg_6mo,num_75_avg_6mo,num_985_avg_6mo,num_100_avg_6mo,num_unq_avg_6mo,total_secs_avg_6mo,count_6mo
count,992931.000000,992931.000000,877161.000000,877161.000000,992931.000000,877161.000000,990834.000000,990834.000000,9.908340e+05,9.908340e+05,...,8.089580e+05,808958.000000,739988.000000,739988.000000,739988.000000,739988.000000,739988.000000,739988.000000,739988.000000,739988.000000
mean,496465.000000,0.063923,5.882329,13.453973,0.581209,6.911228,0.881059,0.887441,2.017018e+07,2.016975e+07,...,6.577121e+03,49.352563,5.817982,1.484480,0.901312,0.967890,24.123430,24.853261,6451.712292,48.739919
std,286634.634404,0.244616,6.423447,20.226865,0.785405,1.904441,0.321801,0.316053,2.199563e+03,2.027108e+03,...,6.115475e+03,28.807377,7.217060,1.843866,0.962576,1.360205,24.349655,19.163324,5940.711577,28.696338
min,0.000000,0.000000,1.000000,-3152.000000,0.000000,3.000000,0.000000,0.000000,1.970010e+07,2.015010e+07,...,3.000000e-02,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.023000,1.000000
25%,248232.500000,0.000000,1.000000,0.000000,0.000000,7.000000,1.000000,1.000000,2.017021e+07,2.017011e+07,...,3.201129e+03,24.000000,2.117647,0.588235,0.400000,0.407407,10.727273,13.032258,3077.988363,23.000000
50%,496465.000000,0.000000,1.000000,0.000000,0.000000,7.000000,1.000000,1.000000,2.017022e+07,2.017012e+07,...,5.017495e+03,51.000000,3.916667,1.032967,0.692308,0.712500,17.761905,20.157895,4882.116562,50.000000
75%,744697.500000,0.000000,13.000000,27.000000,1.000000,9.000000,1.000000,1.000000,2.017023e+07,2.017013e+07,...,7.865384e+03,76.000000,7.062500,1.807692,1.126437,1.183099,28.852941,30.703704,7727.208325,75.000000
max,992930.000000,1.000000,22.000000,2016.000000,2.000000,13.000000,1.000000,1.000000,2.017033e+07,2.017013e+07,...,1.213649e+06,93.000000,1437.978495,302.000000,199.833333,224.156250,2223.428571,926.357143,529534.731357,93.000000


In [7]:
train[['msno','is_churn']].groupby(['is_churn'], as_index=False).count()

,is_churn,msno
0,0,929460
1,1,63471


'0' denotes clients who renews their service and '1' those who churn. Approximately, only **6.82%** of customers churn after the expiration of their subscription.

## Feature selection (by human understanding)

This section covers the process of feature selection but only based on human understanding i.e. which features deemed reasonable to be removed manually, without considering any indicator of feature importance.

In [8]:
Y = train['is_churn'] # extract the label variables

In [9]:
# features to remove
to_rem = ['Unnamed: 0', 'msno', 'is_churn', 'membership_expire_date_last', 'transaction_date_last']

# please correct me if I'm wrong but I don't think records about transaction date would have important effect on the prediction, 
# except the one of TimeSinceReg
X = train.drop(to_rem, axis=1) # remove no., msno, is_churn from data

## Feature transformation (TimeSinceReg) 

In [10]:
# extract the number of days from the attribute TimeSinceReg
# at first the data was '4000 days 00:00:00.000000...'
# at the end, we only need the concrete day such as 4000 --> replace the original data of TimeSinceReg by the number of days

regexp = re.compile('(-?[0-9]+)')
tmp = []
for t in train['TimeSinceReg']:
    if type(t) is not str:
        tmp.append(0)
        continue
    result = regexp.match(t)
    tmp.append(int(result.group(0)))

X['TimeSinceReg'] = tmp

After this step, all data of ambiguous type *object* (msno and TimeSinceRef) have been either removed or transformed into another type appropriate for machine learning method.

## Lacunar features

In [11]:
X.isnull().any(axis=1).sum() # count the total number of rows that have one or more null values

296529

There are 296 529 rows that have null values on one or more attributes. Approximately, these rows occupy **29.86%** over the entire dataset.

## Correlated features

In [12]:
# the list of all categorical features
cat_feat = [
    "city",
    "bd",
    "gender", 
    'registered_via', 
    'is_auto_renew_median', 
    'is_auto_renew_last',
    'plan_list_price_mean', 
    'plan_list_price_last',
    'is_cancel_mean','is_cancel_last']

In [13]:
X_num = X.drop(cat_feat, axis=1) # extract the training set that contains only numerical features

In [ ]:
# libraries to do pretty plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Seaborn style
sns.set_style("whitegrid")

In [ ]:
# Getting correlation matrix
cor_matrix = X_num.corr().round(2)

# Plotting heatmap 
fig = plt.figure(figsize=(20,20));
sns.heatmap(cor_matrix, annot=True, center=0, cmap = sns.diverging_palette(250, 10, as_cmap=True), ax=plt.subplot(111));
plt.show()

The correlated features can be easily deduced from this correlation matrix. Said features are listed as follows (threshold >= 0.8):
* payment_plan_days_mean <==> payment_plan_days_last, actual_amound_paid_mean, actual_amount_paid_last
* payment_method_id_mean <==> payment_method_id_last
* num_25_avg_1mo <==> num_25_avg_3mo
* num_50_avg_1mo <==> num_50_avg_3mo
* num_75_avg_1mo <==> num_75_avg_3mo
* num_985_avg_1mo <==> num_985_avg_3mo
* num_100_avg_1mo <==> num_100_avg_3mo (there is an interesting pattern between those **1mo** and **3mo**), total_secs_avg_1mo, total_secs_avg_3mo
* num_unq_avg_1mo <==> total_secs_avg_1mo, num_unq_avg_3mo
* count_1mo <==> count_3mo
* num_100_avg_3mo <==> total_secs_avg_3mo
* num_unq_avg_3mo <==> num_unq_avg_6mo
* total_secs_avg_3mo <==> total_secs_avg_6mo
* num_100_avg_6mo <==> num_unq_avg_6mo, total_secs_avg_6mo
* num_unq_avg_6mo <==> total_secs_avg_6mo

A quick look at the number of distinct values for categorical variables:

In [ ]:
for v in cat_feat:
    print('%s has %d unique values' % (v, len(X[v].unique())))